### import library

In [1]:
from selenium import webdriver
import requests as r
import pandas as pd
from time import time, sleep
from IPython.core.display import clear_output
from warnings import warn
from selenium.common.exceptions import NoSuchElementException
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.common.by import By
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.action_chains import ActionChains
import re
from datetime import datetime

In [10]:
path = r"C:\Users\Dwi Nissa\AppData\Local\Programs\Python\Python38\Lib\site-packages\selenium\webdriver\firefox\geckodriver.exe"
options = webdriver.FirefoxOptions()
options.set_preference("dom.push.enabled", False)
driver = webdriver.Firefox(executable_path = path, firefox_options=options)
keyword = "tabrak"
radar = ['radarmadura','radarbanyuwangi','radartulungagung','radarjember','radarsurabaya','radarjombang',
                'radarkediri','radarbojonegoro','radarmadiun','radarmalang','radarmojokerto','radarbromo','radarsidoarjo']
url = "https://www.jawapos.com/?s="+keyword
news = [];

r = driver.get(url)
sleep(2)

start_time = time(); req = 0;a = 0;

for _ in range(0,7):     
    #button = driver.find_element_by_css_selector('.load-more').click()
    
    wait = WebDriverWait(driver, 10)
    element = wait.until(EC.element_to_be_clickable((By.CSS_SELECTOR, "a.load-more")))
    element.click()
    print('button pressed! {}'.format(_))
    sleep(2)
    
# Scrap
articles = driver.find_elements(By.XPATH,"/html/body/div[2]/main/div/div/div[1]/section[2]/div[2]/div[1]/div")
links = []
titles = []
datetimes = []
locations = []
for article in articles:
    try:
        link = article.find_element_by_css_selector('.d-block').get_attribute('href')
        result = re.search('.*://(.*).jawapos.com', link)
        # check wheter link is one of east java's radar
        if result.group(1) in radar:
            # if yes, then append to array
            links.append(link)
            try:
                titles.append(article.find_element_by_css_selector('.post-list__title').text)
            except(NoSuchElementException, IndexError, ValueError):
                pass
            try:
                datetimes.append(article.find_element_by_css_selector('.post-list__time').text)
                print(article.find_element_by_css_selector('.post-list__time').text)
            except(NoSuchElementException, IndexError, ValueError):
                pass
            try:
                locations.append(article.find_element_by_css_selector('.category').text)
            except(NoSuchElementException, IndexError, ValueError):
                pass
    except(NoSuchElementException, IndexError, ValueError):
        pass

clear_output(wait = True);
print("Getting ready scrap {} of news items....".format(len(links)))
    
for i in range(len(links)):
    content = ""
    try:
        driver.get(links[i])
        sleep(2)

        try:
            content = driver.find_elements(By.XPATH,"//div[@class='tdb-block-inner td-fix-index']")[15].text
        except(NoSuchElementException, IndexError, ValueError):
            pass
        # Telah disesuaikan untuk semua website radar di jawapos
        if content=="":
            try:
                content = driver.find_element_by_xpath("//div[@itemprop='articleBody']").text
            except(NoSuchElementException, IndexError, ValueError):
                pass
        if content=="":
            try:
                content = driver.find_element_by_xpath("//div[@dir='auto']").text
            except(NoSuchElementException, IndexError, ValueError):
                pass
        if content=="":
            try:
                content = driver.find_element_by_xpath("//div[@class='entry-content entry clearfix']").text
            except(NoSuchElementException, IndexError, ValueError):
                pass
        if content=="":
            try:
                contents = [con.text for cont in driver.find_elements_by_xpath("//div/p[@style='text-align: justify']")]
                content = ' '.join(contents)
            except(NoSuchElementException, IndexError, ValueError):
                pass
        if content=="":
            try:
                contents = [cont.text for cont in driver.find_elements_by_xpath("//div[@class='paragraph-article-detail']/p")]
                content = ' '.join(contents)
            except(NoSuchElementException, IndexError, ValueError):
                pass
        news.append([links[i],datetimes[i],locations[i],titles[i],content])
    except:
        pass

print('-----------------------FINISHED-----------------------')
print('--------Scrap Duration: {} seconds.--------'.format(start_time-time()))


Request:31; Frequency:0.06585783355646777 req/sec
Request:32; Frequency:0.06676812230213629 req/sec
Request:33; Frequency:0.06601364482149553 req/sec
Request:34; Frequency:0.06681544302296652 req/sec
Request:35; Frequency:0.06785461158609779 req/sec
Request:36; Frequency:0.06861945019784782 req/sec
Request:37; Frequency:0.06569312619523844 req/sec
Request:38; Frequency:0.06671751370034938 req/sec
-----------------------FINISHED-----------------------
--------Scrap Duration: -569.5885100364685 seconds.--------


In [11]:
data = pd.DataFrame(news, columns=['Link','Date Time','Location','Title','Content'])
data

,Link,Date Time,Location,Title,Content
0,https://radarsurabaya.jawapos.com/read/2021/02...,"9 Februari 2021, 02:10:59 WIB",SURABAYA,"Tabrak Pintu Mobil yang Dibuka Mendadak, Pemot...",SURABAYA - Ini peringatan bagi pengemudi mobil...
1,https://radarbromo.jawapos.com/pasuruan/08/02/...,"8 Februari 2021, 07:30:14 WIB",JAWA TIMUR,"Mau ke Bali, Pemotor asal Klaten Tabrak Truk d...","Desktop_AP_Leaderboard 1\nKRATON, Radar Bromo ..."
2,https://radarbromo.jawapos.com/pasuruan/08/02/...,"8 Februari 2021, 04:00:24 WIB",JAWA TIMUR,Siswa TK Tewas Jadi Korban Tabrak Lari di Bukir,"Desktop_AP_Leaderboard 1\nGADINGREJO, Radar Br..."
3,https://radarbromo.jawapos.com/pandaan/07/02/2...,"7 Februari 2021, 16:58:34 WIB",JAWA TIMUR,"Pikap Tabrak Warung di Sukorejo, Penjaga di Da...","Desktop_AP_Leaderboard 1\nSUKOREJO, Radar Brom..."
4,https://radarbromo.jawapos.com/pasuruan/05/02/...,"5 Februari 2021, 22:10:43 WIB",JAWA TIMUR,"Terburu Karena Anak Sakit, Pikap Tabrak Tiang ...","Desktop_AP_Leaderboard 1\nWONOREJO,Radar Bromo..."
5,https://radarbromo.jawapos.com/pandaan/26/01/2...,"26 Januari 2021, 10:48:28 WIB",JAWA TIMUR,"Rem Dump Truk Ini Blong, Tabrak Papan Penunjuk...","Desktop_AP_Leaderboard 1\nPURWOSARI, Radar Bro..."
6,https://radarbromo.jawapos.com/pandaan/26/01/2...,"26 Januari 2021, 10:48:28 WIB",JAWA TIMUR,"Rem Dump Truck Ini Blong, Tabrak Papan Penunju...","Desktop_AP_Leaderboard 1\nPURWOSARI, Radar Bro..."
7,https://radarsurabaya.jawapos.com/read/2021/01...,"25 Januari 2021, 02:40:47 WIB",SURABAYA,"Motor Tabrak Jembatan, Emak-Emak Tewas Tercebu...","SIDOARJO-Isti Khomah, 43, warga Desa Panggreh ..."
8,https://radarbromo.jawapos.com/kraksaan/24/01/...,"24 Januari 2021, 19:11:35 WIB",JAWA TIMUR,"Sopir Mengantuk, Truk Tabrak Median Jalan-PJU ...","Desktop_AP_Leaderboard 1\nKRAKSAAN, Radar Brom..."
9,https://radarsurabaya.jawapos.com/read/2021/01...,"24 Januari 2021, 06:12:08 WIB",SURABAYA,"Ngebut dan Ngantuk, Mahasiswa Tewas Tabrak Pohon",SURABAYA-Kecelakaan lalu lintas tunggal menyeb...


In [12]:
data.to_csv("data/update-jawapos_tabrak.csv")